In [1]:
!pip install pandas-bokeh > /dev/null
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

## Introduction

This is a Playo data exploration that I did for fun with some data I have been scraping for about 4 years now, for [this tool](https://punchagan.github.io/playo-find-venue/) that I built to find a Playo venue quickly. I have been scraping data for Bangalore, Hyderabad and Chennai. And I wanted to see if there's something interesting I'd find from the data. 

My initial curiosity was to see if the data would let me see the impact of COVID on playing patterns. But, the data isn't rich enough, and I wasn't very sure I'd be able to see much. But, it was worth a try.  I start by exploring the data and trying to understand it, and proceeding to see if I find any thing interesting in the data.

In [2]:
import pandas as pd
import functools
import datetime
from IPython.display import display, Markdown, Latex

from datetime import datetime as dt

from bokeh.models import Span, Label
from bokeh.plotting import show

In [3]:
# Infer missing data columns from info column
# FIXME: Data scraping could've been better, to avoid this headache!

# Notably, there's no reliable ID for each of the entries. 
# We can possibly use the link as an ID to identify a venue across different dates
def get_ratings_and_link(text):
    for line in text.splitlines():
        if 'Ratings:' in line:
            _, rating, count = line.split()
            count = count.strip('[').split(']')[0]
        if 'href' in line:
            link = line.split('"')[1]
            
    return (rating, count, link)
        
def append_missing_columns(data):
    maybe_missing = ('avgRating', 'ratingCount', 'deferLink')
    missing_columns = [x not in data.columns for x in maybe_missing]
    if any(missing_columns):
        info = data.loc[:, ['info']].applymap(get_ratings_and_link)
        missing = pd.DataFrame(info['info'].values.tolist(), index=info.index, 
                               columns=['avgRating', 'ratingCount', 'deferLink'])
        data = pd.concat([data, missing.loc[:, missing_columns]], axis=1)
        data['avgRating'] = pd.to_numeric(data['avgRating'], downcast='float', errors='coerce')
        data['ratingCount'] = pd.to_numeric(data['ratingCount'], downcast='integer', errors='coerce')
    return data

In [4]:
import os
import glob
DATA_DIR = '../../playo-find-venue/snapshots'
SNAPSHOTS = glob.glob(f"{DATA_DIR}/*.json")
CITIES = sorted({os.path.basename(snapshot).split('_', 2)[1] for snapshot in SNAPSHOTS})

In [5]:
DATA = {}
for city in CITIES:
    snapshots = glob.glob(f"{DATA_DIR}/venues_{city}*.json")
    snapshots = sorted(snapshots)
    for snapshot in snapshots:
        data = pd.read_json(snapshot)
        city, date = tuple(os.path.basename(snapshot).split('.')[0].split('_')[1:])
        month = date.rsplit('-', 1)[0]
        try:
            DATA[(city, date)] = append_missing_columns(data)
        except KeyError:
            print(snapshot)

In [6]:
def get_latest(city):
    assert city in CITIES
    latest_key = sorted(key for key in DATA.keys() if key[0] == city)[-1]
    return DATA[latest_key]

In [7]:
keys = {city: [key for key in DATA.keys() if key[0] == city] for city in CITIES}

## Number of Venues by City

Let's start with looking at the number of venues by city. 

In [8]:
counts = [pd.DataFrame.from_dict({key[1]: DATA[key].shape[0] for key in keys[city]}, orient='index', columns=[city]) for city in CITIES]

In [9]:
def mark_waves(fig):
    label = Label(x=datetime.datetime(2020, 4, 1), y=100, y_units='screen',
                  text='1st COVID Wave',
                  angle=90, angle_units='deg')
    fig.add_layout(label)
    label = Label(x=datetime.datetime(2021, 4, 1), y=100, y_units='screen',
                  text='2nd COVID Wave', 
                  angle=90, angle_units='deg')
    fig.add_layout(label)

    wave1 = Span(
        location=dt(2020, 4, 1),
        dimension='height', 
        line_color='#009E73',
        line_dash='dashed', 
        line_width=3
    )
    fig.add_layout(wave1)

    wave2 = Span(
        location=dt(2021, 4, 1),
        dimension='height', 
        line_color='#F0E442',      
        line_dash='dashed', 
        line_width=3
    )
    fig.add_layout(wave2)

def merge_and_plot(data, ylabel, show_waves=False):
    merged = functools.reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True, how='outer'), data)
    merged.index = [datetime.datetime.strptime(d, '%Y-%m-%d') for d in merged.index]
    fig = merged.plot_bokeh(
        kind='line', 
        plot_data_points=True,
        plot_data_points_size=3,
        xlabel='Dates',
        ylabel=ylabel,
        sizing_mode="stretch_width",
        legend='top_left',
        vertical_xlabel=True,
        show_figure=not show_waves,
        zooming=False,
    )
    if show_waves:
        mark_waves(fig)
    show(fig)

In [10]:
_ = merge_and_plot(counts, 'Venues')

I know from playing and using the app in Hyderabad and Bangalore that Playo is much more popular and has lot more venues in Bangalore than Hyderabad. So, this graph is a little surprising. Also, the spike in May 2019, seems a little fishy.

### What happened in May 2019?

I think Playo added a bunch of public sports venues to their database, in an attempt to just display them on their app (even if not bookable) or to allow some kind of mechanism for booking in future... But, most of these venues added in that period don't seem to have any other "usage" data in the current snapshot of data.

Looking at only the venues which have at least one user rating would make things a little bit clearer. 

In [11]:
def uniq_lat_lng(data):
    return set((lat, lng) for (lat, lng) in data[['lat', 'lng']].values)

def current_and_rated(data, city, only_current=False):
    current = {city: uniq_lat_lng(get_latest(city)) for city in CITIES}
    data_f = data[data.apply(lambda x: (x.lat, x.lng) in current[city], axis=1)]
    return data_f[data_f['ratingCount'].notna()] if only_current else data[data['ratingCount'].notna()]

ratedCounts = [
    pd.DataFrame.from_dict(
        {key[1]: current_and_rated(DATA[key], city, only_current=False).shape[0] for key in keys[city]}, 
        orient='index', 
        columns=[city]
    ) 
    for city in CITIES
]
_ = merge_and_plot(ratedCounts, 'Venues with atleast one rating (as of that day)')

In [12]:
def get_month_changes(city, year_month):
    year, month = year_month
    month_ = month + 1
    year_ = year
    if month_ % 13 == 0:
        month_ = 1
        year_ = year + 1
    next_ = f'{year_}-{month_:02}-01'
    date = f'{year}-{month:02}-01'

    end = DATA[(city, next_)]
    start = DATA[(city, date)]
    end_lat_lng = uniq_lat_lng(end)
    start_lat_lng = uniq_lat_lng(start)
    added = end_lat_lng - start_lat_lng
    removed = start_lat_lng - end_lat_lng
    A = end[end.apply(lambda x: (x.lat,x.lng) in added, axis=1)]
    R = start[start.apply(lambda x: (x.lat, x.lng) in removed, axis=1)]
    return A, R
    
added, _ = get_month_changes('bangalore', (2019, 5))
added.plot_bokeh(
    kind='map', 
    y='lat', 
    x='lng', 
    legend=False, 
    title='New Venues added in May 2019', 
    return_figure=False,
    figsize=(800, 600),
    zooming=False,
)
added[added['ratingCount'].notna()].plot_bokeh(
    kind='map', 
    y='lat', 
    x='lng', 
    legend=False, 
    title='New Venues with at least one rating', 
    return_figure=False, 
    sizing_mode="stretch_width",
    figsize=(800, 600),
    zooming=False,
)

## Number of ratings by City

We use number or ratings as a proxy for number of users... though it's such a tricky measure. 

In [13]:
ratingCounts = [pd.DataFrame.from_dict({key[1]: DATA[key]['ratingCount'].sum() for key in keys[city]}, orient='index', columns=[city]) for city in CITIES]

In [14]:
_ = merge_and_plot(ratingCounts, 'Total Ratings (all venues)')

In [15]:
ratingCounts = [pd.DataFrame.from_dict({key[1]: DATA[key]['ratingCount'].sum() for key in keys[city]}, orient='index', columns=[city]) for city in CITIES]
monthlyCounts = [r[1:] - r[:-1].values for r in ratingCounts]
_ = merge_and_plot(monthlyCounts, 'Total Ratings (Monthly Change)')

Why do number of ratings go down?

- You can't remove your rating, you can only vote once for a venue. It's not linked to whether you actually played in that venue or not. I think there was a problem with owners voting for their own venue or -ve voting competition or something.

- Did some venues get removed, that got rid of ratings? Yes, that is indeed the case. We can guess at that from the number of venues plot above, but also by looking at the specific changes in data.

In [16]:
added, removed = get_month_changes('bangalore', (2021, 8))
display(Markdown(f"For instance, ~{len(removed)} venues got removed in Bangalore in August 2021"))

For instance, ~50 venues got removed in Bangalore in August 2021

So, ratings graph above doesn't give a sense of users, even if ratings are a bad proxy for usage given that a user can only vote once for a venue.  To improve the situation slightly, we can look at ratings for venues that are only still on the listing as of today, and see how ratings on those venues changed.

In [17]:
current = {city: uniq_lat_lng(get_latest(city)) for city in CITIES}

currentRatingCounts = [
    pd.DataFrame.from_dict(
        {
            key[1]: DATA[key][DATA[key].apply(lambda x: (x.lat, x.lng) in current[city], axis=1)]['ratingCount'].sum() 
            for key in keys[city]
        }, 
        orient='index', 
        columns=[city]
    ) 
    for city in CITIES
]
merge_and_plot(currentRatingCounts, 'Ratings for venues that are still on Playo', show_waves=True)

cMonthlyCounts = [r[1:] - r[:-1].values for r in currentRatingCounts]
merge_and_plot(cMonthlyCounts, 'Total Ratings (Monthly Change) for venues still on Playo', show_waves=True)

NOTE: The data wasn't scraped correctly -- the IDs of the venues weren't stored in the data. So, it's a little hackish to track a venue over time. We could use the link as the ID, but links could change too. We use (lat, lng) as an ID, because they are less likely to change and would change only in case of errors. 

# Couple of Venues that I play at regularly

In [18]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.ranges import Range1d
from bokeh.models import LinearAxis, Range1d

def find_venue_over_time(name, city, show_waves=True):
    current = get_latest(city)
    venue = current[current['name'] == name].iloc[0]
    lat, lng = venue.lat, venue.lng
    data = []
    for key in keys[city]:
        d = DATA[key][(DATA[key].lat == lat) * (DATA[key].lng == lng)]
#         print(d)
#         print(len(d))
        if len(d) == 1:
            d.index = [datetime.datetime.strptime(key[1], '%Y-%m-%d')]
            data.append(d)
    data = pd.concat(data)
    fig = data['ratingCount'].plot_bokeh(
        legend='bottom_right', 
        show_figure=False, 
        zooming=False, 
        title=f"{name} ({city.capitalize()})",
        vertical_xlabel=True,
        sizing_mode="stretch_width",
        xlabel='Dates',
    )
    fig.extra_y_ranges = {"avgRatings": Range1d(start=0, end=5)}
    fig.add_layout(LinearAxis(y_range_name="avgRatings", axis_label='rating'), 'right')
    fig.line(
        data.index.values, data['avgRating'], 
        y_range_name='avgRatings', 
        legend_label="rating",
        name='rating',
        color='red',
        dash='dashed',
    )
    
    if show_waves:
        mark_waves(fig)


    show(fig)
    display(Markdown(f"[{name} on Playo]({venue.deferLink})"))

name, city = 'Pulse 7', 'hyderabad'
find_venue_over_time('Pulse 7', 'hyderabad')
find_venue_over_time('Smash O Station (SOS)', 'bangalore')

[Pulse 7 on Playo](https://z34v4.app.goo.gl/nWs5)

[Smash O Station (SOS) on Playo](https://z34v4.app.goo.gl/ZPxi)

# Venues with most ratings by City

In [19]:
for city in CITIES:
    latest = get_latest(city)
    venue = latest.loc[latest['ratingCount'].idxmax()]
    find_venue_over_time(venue['name'], city)

[Orchid Sports Academy on Playo](https://z34v4.app.goo.gl/W1Zx)

[AJS Sports Arena on Playo](https://z34v4.app.goo.gl/MDNL)

[Elite Sports Hub on Playo](https://z34v4.app.goo.gl/gWYv)

# Which venues got most ratings since July 2021?

To try and see how Covid affected playing, we plotted the graphs above with total rating counts across a city, and ratings for some specific venues. We can see some flat portions on the graphs around the time of 1st and 2nd waves, but most of these numbers already seem to be plateauing to be able to notice anything significant, really. 

To see if we can notice something more significant, let's look at the venues that seem to be growing/upcoming and see how they were affected.

In [20]:
currentData = {
    key: DATA[key][DATA[key].apply(lambda x: (x.lat, x.lng) in current[city], axis=1)] 
    for city in CITIES for key in keys[city]
}

In [21]:
latest = DATA[('bangalore', '2022-02-01')]
start = DATA[('bangalore', '2021-07-01')]
#start = DATA[('bangalore', '2020-03-01')]
merged = pd.merge(left=latest, right=start, on=('lat', 'lng'), how='inner', suffixes=['_curr', '_init'])
merged['countDiff'] = merged['ratingCount_curr'] - merged['ratingCount_init']
name_changes = ('Racket Club', 'ARATT', 'DHI Sports', 'JustPlay', 'Sports Park', 'Rio Sports', 'Astro Arena')
filter_by_hand = merged.name_curr.str.startswith(name_changes) ^ True
top = merged[filter_by_hand][['name_curr', 'name_init', 'ratingCount_curr', 'ratingCount_init', 'countDiff']].sort_values('countDiff', ascending=False)[:10]
top

,name_curr,name_init,ratingCount_curr,ratingCount_init,countDiff
163,Eesha Badminton Academy,Eesha Badminton Academy,116.0,33.0,83.0
36,Aptha Badminton Academy,Aptha Badminton Academy,236.0,194.0,42.0
413,Prakash Badminton Academy,Prakash Badminton Academy,58.0,20.0,38.0
351,NRC Badminton Arena,NRC Badminton Arena,95.0,57.0,38.0
591,The Majesstine Sports,The Majesstine Sports,160.0,123.0,37.0
403,Play Zone - Kasturinagar,Play Zone - Kasturinagar,204.0,172.0,32.0
474,Score Bengaluru Sports Park,Score Bengaluru Sports Park,150.0,126.0,24.0
387,Panchajanya Badminton & Fitness Academy,Panchajanya Badminton & Fitness Academy,103.0,82.0,21.0
367,Nova Badminton Academy - Arehalli,Nova Badminton Academy - Arehalli,237.0,216.0,21.0
656,Whitefield United - Cap Life,Whitefield United,43.0,23.0,20.0


In [22]:
eesha = """
The venue seems to have gotten about 30 votes in the first month of being added on PlayO (Apr 2021). And the ratings were pretty high. I'm guessing these were ratings by friends of the venue owners, because I remember going to the venue some time in July/August of 2021 and being utterly disappointed by it, which is reflected in the sudden drop in ratings, as soon as the lockdowns in Bangalore eased a little after the second wave. 
"""
nrc = """Not really sure what happened here. There was a huge spike in the number of ratings, taking the avgRating down drastically, before the number of ratings seems to have been reset. I wonder if there were some bad actors at play?
"""
for name in top.name_curr:
    find_venue_over_time(name, 'bangalore')
    if name == 'Eesha Badminton Academy':
        display(Markdown(eesha))
    elif name == 'NRC Badminton Arena':
        display(Markdown(nrc))

[Eesha Badminton Academy on Playo](https://z34v4.app.goo.gl/3FKK)


The venue seems to have gotten about 30 votes in the first month of being added on PlayO (Apr 2021). And the ratings were pretty high. I'm guessing these were ratings by friends of the venue owners, because I remember going to the venue some time in July/August of 2021 and being utterly disappointed by it, which is reflected in the sudden drop in ratings, as soon as the lockdowns in Bangalore eased a little after the second wave. 


[Aptha Badminton Academy on Playo](https://z34v4.app.goo.gl/ncJe)

[Prakash Badminton Academy on Playo](https://z34v4.app.goo.gl/9Esk)

[NRC Badminton Arena on Playo](https://z34v4.app.goo.gl/cfgi)

...

[Play Zone - Kasturinagar on Playo](https://z34v4.app.goo.gl/mXUF)

[Score Bengaluru Sports Park on Playo](https://z34v4.app.goo.gl/GVXB)

[Panchajanya Badminton & Fitness Academy on Playo](https://z34v4.app.goo.gl/y2bF)

[Nova Badminton Academy - Arehalli on Playo](https://z34v4.app.goo.gl/oyAE)

[Whitefield United - Cap Life on Playo](https://z34v4.app.goo.gl/nhxs)

In [23]:
!pip install pydeck > /dev/null

# Just playing around with Deck.gl 

Height of the skyscrapers is proportional to the number of ratings a venue has. 

Zoom in and hover over the venues to see more info about the venue.

Pan around to Hyderabad and Chennai to see how they compare to Bangalore 

In [27]:
import pandas as pd
import pydeck as pdk

data = [get_latest(city) for city in CITIES]
df = pd.concat(data)
df['ratingAvg'] = df['avgRating'].apply('{:.2f}'.format)
color_lookup = pdk.data_utils.assign_random_colors(df['rating'])
df['color'] = df['rating'].apply(lambda x: color_lookup.get(str(x)))
view = pdk.data_utils.compute_view(data[0][["lng", "lat"]], view_proportion=0.8)
view.pitch = 90
view.bearing = 60

layer = pdk.Layer(
    "ColumnLayer",
    data=df,
    get_position=["lng", "lat"],
    radius=40,
    elevation_scale=40,
    get_elevation='ratingCount',
    get_fill_color='color',
    pickable=True,
    extruded=True,
)

tooltip = {
    "html": '<b>{name}</b> ({ratingAvg} [{ratingCount}]) <br>{filter_by}',
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},
}

r = pdk.Deck(
    layer,
    initial_view_state=view,
    tooltip=tooltip,
    map_provider="mapbox",
    map_style=pdk.map_styles.MAPBOX_LIGHT,
)
r.to_html()

In [28]:
from IPython.display import Javascript

script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
Javascript(script)

<IPython.core.display.Javascript object>

In [26]:
# Export to data projects as html
!jupyter nbconvert analysis.ipynb --to html --no-input --output=index --embed-images >/dev/null 2>&1
!firefox index.html